In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob, sys, h5py
import seaborn as sns
sys.path.append('/Users/nneveu/github/pyssnl/')
# sys.path.append('/Users/nneveu/github/libensemble/')
sys.path.append('/Users/nneveu/github/emittance_minimization/code/')
sys.path.append('/gpfs/slac/staas/fs1/g/accelerator_modeling/nneveu/emittance_minimization/code/')

from libeopal import LibeOpal
from libeopal import opal_deap

from astra import Astra, template_dir
import distgen, ssnl
from distgen import Generator
from distgen.writers import *
from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.plot import marginal_plot

import ssnl 

plt.rc('xtick',labelsize=12)
plt.rc('ytick',labelsize=12)
plt.rc('axes',labelsize=14)
plt.rc('axes', axisbelow=True)


In [ ]:
TOP_DIR = '/Users/nneveu/github/emittance_minimization/code/slac/'
sim_dir  = '/Users/nneveu/github/emittance_minimization/code/slac/'
files = 'ssnl/'
# temp_dir = '/Users/nneveu/github/beam-shaping/sfg/'
# sim_dir  = '/Users/nneveu/github/beam-shaping/sfg'

# Make ssnl dist
# ssnl_file = temp_dir+'tlaser_dist.yaml'
ssnl_file = TOP_DIR+'tlaser_dist.yaml'

In [ ]:
STAT_NAMES = ['t', 's','numParticles','charge','energy','rms_x', 'rms_y', 'rms_s', \
              'rms_px', 'rms_py', 'rms_ps', 'emit_x', 'emit_y', 'emit_s']#, 'mean_x', \


xscale   = np.array([('RADIUS',1e-2), ('GDD', 1.0), ('SF', 0.1), \
                    ('PHGUNB', 1.0), ('PHBUN', 1.0), ('GBUN',0.1), \
                    ('SF1', 1e-3), ('SF2', 1e-3), \
                    ('PHCM1', 1.0), ('GCM1', 1.0),\
                    ('PHCM2', 1.0), ('GCM2', 1.0),\
                    ('PHCM3', 1.0), ('GCM3', 1.0),\
                    ('PHCM4', 1.0), ('GCM4', 1.0),\
                   ],dtype=[('name', 'U10'), ('scale', 'f4')])

xbounds  = np.array([('RADIUS', 15.0, 75.0), ('GDD', 1.0, 30.0), ('SF',0.0, 12.5),
                    ('PHGUNB', -20.0, 10.0), ('PHBUN', -100.0, -10.0), ('GBUN', 10.0, 35.0), \
                    ('SF1', 20.0, 70.0), ('SF2', 20.0, 70.0), \
                    ('PHCM1', -40.0, 40.0), ('GCM1', 0.0, 32.0), \
                    ('PHCM2', -40.0, 40.0), ('GCM2', 0.0, 32.0), \
                    ('PHCM3', -40.0, 40.0), ('GCM3', 0.0, 32.0), \
                    ('PHCM4', -40.0, 40.0), ('GCM4', 0.0, 32.0), \
                   ],dtype=[('name', 'U10'), ('lb', 'f4'), ('ub', 'f4')])


objscale =  np.array([
        ('emit_x',0.3e-6,6e-6),
        ('rms_s',0.5e-3,4e-3),
        ],dtype=[('name', 'U10'), ('lb', 'f4'), ('ub', 'f4')])

# Keys related to data in OPAL sim and objectives
key_dict = {'data_keys': STAT_NAMES,
            'dvar_keys': list(xbounds['name'][:]),
            'objective_keys':objscale['name']}


In [ ]:
x = np.array( [ 54.15268923,  23.09526061,  9, -14.28070143, -67.43214943, \
   27.08847979,  56.00315678,  33.93138405,  10.73151142,  10.12314852, \
  -25.63778875,  13.84402314,  36.53945811,   5.11763022, -22.81990719, \
   15.68010058])

#State the objective function, its arguments, output, and necessary parameters (and their sizes)
sim_specs = {'sim_f': opal_deap, #fitness_opal, # This is the function whose output is being minimized
             'in': ['individual'],# These keys will be given to the above function
             'out': [('fitness_values', float, 2)]+ [(key,float) for key in key_dict['data_keys']] + [(key+'_long',float, 2500) for key in key_dict['data_keys']],
              'user': {'key_dict': key_dict,
                       'basefile_name': 'sc_inj_C1',
                       'input_files_path':TOP_DIR+files,
                       'distgen_file':TOP_DIR+files+'tlaser_dist.yaml',
                       'zstop': 14.95,
                       'penalty_scale': 20,
                       'xscales':xscale,
                       'objective_scales':objscale,
                       'cores': 4,
                       'sim_particles':50000,
                       'sim_kill_minutes': 10.0,
                       'laser_filter': 0.5,
                       } # end user specs
            }# end sim specs


osim  = LibeOpal(sim_specs)
#osim.run_sim(x)

distribution = osim.make_ssnl_dist(x)

In [ ]:
part = distribution.particles

# x = plt.hist2d(part['t']*10**3, part['pz']*10**3, 100, cmap='YlGnBu') #facecolor='blue')
sns.histplot(part['t']*10**12, color='indigo', stat='count', bins=500, kde=True)
plt.show()

In [ ]:
xrad = x[0]
xgdd = x[1]
xsf  = x[2]

xopal = x[3:]

BW  = 0.5
a   = ssnl.SSNL()
u   = ssnl.UNITS()
gdd = xgdd*1000
sf  = xsf*xscale['scale'][2] 

a.set_default(gdd, sf, 1024, 246)
a.genEqns()
a.genGrids()
a.genFields()
a.propagate()

time, uv = a.generate_uv(BW)
a.plot_green_uv(BW)

In [ ]:
# sxscale   = np.array([('RADIUS',1e-2), ('GDD', 1.0), ('SF', 0.1), ('PHGUNB', 1.0), ('PHBUN', 1.0), ('GBUN',0.1), ('SF1', 1e-3), ('SF2', 1e-3), \
#                     ('PHCM1', 1.0), ('GCM1', 1.0), \
#                     ('PHCM2', 1.0), ('GCM2', 1.0), \
#                     ('PHCM3', 1.0), ('GCM3', 1.0), \
#                     ('PHCM4', 1.0), ('GCM4', 1.0), \
#                    ],dtype=[('name', 'U10'), ('scale', 'f4')])

# x = np.array( [ 54.15268923,  23.09526061,  10.90621621, -14.28070143, -67.43214943, \
#    27.08847979,  56.00315678,  33.93138405,  10.73151142,  10.12314852, \
#   -25.63778875,  13.84402314,  36.53945811,   5.11763022, -22.81990719, \
#    15.68010058])

In [ ]:
# # Filter green profile, get UV profile 
# green = a.eField['freq'][3][-1,:]
# filt  = np.exp( -( (a.lists['lambda'][2,:] - 515*u.nm) / (BW*u.nm) )**4 );
# green_filtered = green* filt;
# green_ifft     = ssnl.ifft(green_filtered)
# green_intensity= abs(green_ifft**2)
# suv_intensity   = green_intensity**2

# # Tim distribution in ps
# st = np.arange(0,len(suv_intensity))*a.grids['dt']*10**12
# st = st-max(st)/2

# sX  = np.zeros((len(st),2))
# sX[:,0] = st
# sX[:,1] = suv_intensity

# with open(sim_dir+'tdist_file_ssnl.dat', 'wb') as f:
#     np.savetxt(f, sX, fmt = ['%2.9f', '%2.9f'], delimiter=' ', newline='\n')

# sdist = Generator(input=ssnl_file)#, verbose=True)
# sdist.input['r_dist']['max_r']['value'] = xrad*sxscale['scale'][0]
# #sdist.input['start']['MTE']['value'] = 250 
# #sdist.input['n_particle'] = int(1e7)
# sdist.run()
# sparticles = sdist.particles
# #sparticles.write_opal(sim_dir+'opal_emitted_ssnl_10mill.txt', dist_type= 'emitted')

In [ ]:
# num_bins = 100
# plt.figure()
# plt.plot(st, suv_intensity*10**-27, c='teal', alpha=0.75)
# plt.xlim(-20,20)
# #plt.ylim(0,2.0)
# plt.xlabel('Time (ps)', size=14)
# plt.ylabel('Intensity (arb.)', size=14)
# plt.grid()
# #sns.histplot(sparticles["t"]*10**12, color='indigo', stat='count', bins=500, kde=True)
# #plt.hist2d(sparticles['x']*10**3, sparticles['y']*10**3, num_bins, cmap='YlGnBu') #facecolor='blue')
# #plt.xlim(-25,25)
# #plt.ylim(0,320)
# plt.grid()
# plt.savefig('test_ssnl_bw0.5_dist.pdf', dpi=300, bbox_inches='tight')
# plt.show()
